<a href="https://colab.research.google.com/github/hiu04/NLP-Deep-Learning/blob/main/MMAI5400_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMAI 5400 Assignment 1 – Web Scraping

For this assignment you will scrape reviews from Trustpilot (in the second
assignment you will classify the sentiment of those reviews).
Submission
This assignment should be submitted as Python 3 code and uploaded to Canvas.
The submission should be a single PY file, and not a Jupyter Notebook. The
due date is on October 4 at 8:30 am.
The code will be tested and should produce the output specified below.

**Task**

Your task is to scrape reviews from Trustpilot. You chose a company for the
reviews, for example, Skype. Make sure that the company has at least 500
reviews. The reviews should be written to a CSV file with the following columns:
companyName, datePublished, ratingValue, reviewBody.

## Use the requests module to download the html for URL.

In [ ]:
import requests
# Fine URL from Trustpilot page for TD Bank
resp = requests.get('https://www.trustpilot.com/review/tdbank.ca')

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(resp.text, 'html.parser')
# This is the html source
html_code = resp.text

In [ ]:
soup

<!DOCTYPE html>
<html lang="en-US"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="/manifest.json" rel="manifest"/><meta content="Trustpilot" name="application-name"/><meta content="#1c1c1c" name="theme-color"/><link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/><link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link color="#00b67a" href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/safari-pinned-tab.svg" rel="mask-icon"/><meta content="Trustpilot" name="apple-mobile-web-app-title"/><meta content="#1c1c1c" name="msap

## Extract the total number of reviews.

In [ ]:
item = soup.find(
    name='p',
    attrs={'class':'typography_body-l__KUYFJ typography_appearance-default__AAY17'}
)
N = int(item.contents[0].replace(',',''))
print(f'Total number of reviews is {N}.')

Total number of reviews is 1995.


## Iterate over review pages and extract reviews.

In [ ]:
import json
import pandas as pd

BASE_URL = 'https://ca.trustpilot.com'
initial_url = 'https://www.trustpilot.com/review/tdbank.ca'
current_url = initial_url

# Prepare an empty list to hold the extracted data
records = []
count = 0

# Extract 550 comments
comments_num = 550

while count < comments_num:
    # Combine previous codes
    response = requests.get(initial_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the JSON data
    script = soup.find('script', {'data-business-unit-json-ld': 'true'})
    data = json.loads(script.string)

    # Extract information
    for item in data['@graph']:
        if count >= comments_num:
            break

        # Check for 'LocalBusiness' type for company name
        if item['@type'] == 'LocalBusiness':
            companyName = item['name']

        # Check for 'Review' type for review details
        if item['@type'] == 'Review':
            datePublished = item.get('datePublished', 'N/A')
            ratingValue = item['reviewRating']['ratingValue']
            reviewBody = item['reviewBody']

            records.append([companyName, datePublished, ratingValue, reviewBody])
            count += 1

    # Get the next page URL
    next_page_elements = soup.find_all(name="a", string='Next page')
    if not next_page_elements or count >= comments_num:
        break

    initial_url = 'https://ca.trustpilot.com' + next_page_elements[0]['href']

# Convert the list of records into a DataFrame
df = pd.DataFrame(records, columns=["comapnyName", "datePublished", "ratingValue", "reviewBody"])

# Display the table
print(df)

    comapnyName             datePublished ratingValue  \
0       TD Bank  2023-09-28T00:54:59.000Z           1   
1       TD Bank  2023-09-29T00:04:04.000Z           1   
2       TD Bank  2023-09-27T17:53:35.000Z           1   
3       TD Bank  2023-09-26T13:54:51.000Z           5   
4       TD Bank  2023-09-16T16:05:19.000Z           1   
..          ...                       ...         ...   
545     TD Bank  2022-08-26T20:10:55.000Z           1   
546     TD Bank  2022-08-26T19:03:31.000Z           1   
547     TD Bank  2022-08-26T02:19:27.000Z           1   
548     TD Bank  2022-08-26T02:02:35.000Z           1   
549     TD Bank  2022-08-25T23:45:21.000Z           1   

                                            reviewBody  
0    I originally purchased a product from Samsung ...  
1    There should be a zero stars option. Every asp...  
2    The William Kitchen TD bank is close to my hou...  
3    TD is a good bank for my general banking needs...  
4    My sons account was locke

## Clean Extract Data

In [ ]:
# Change the 'datePublished' to date formate (e.g., YYYY-MM-DD)
df['datePublished'] = pd.to_datetime(df['datePublished'])
df['datePublished'] = df['datePublished'].dt.strftime('%m-%d-%Y')

## Save as CSV and .py file

In [ ]:
# Save the dataframe to a CSV file
df.to_csv('TDBank_reviews.csv', index=False)